## Projeto Final:

Nesse projeto nosso objetivo é:
- Buscar termos específicos
- Salvar resultados em um arquivo txt
- Mostrar os resultados sobre a imagens dos termos específicos

### 1. Preparando o ambiente

#### Biblioteca e dados

In [ ]:
!pip install opencv-python==4.6.0
!sudo apt install tesseract-ocr
!pip install pytesseract==0.3.9

In [ ]:
! git clone https://github.com/sthemonica/text-recognize

In [ ]:
import pytesseract
import numpy as np
import cv2
import re
import os
import matplotlib.pyplot as plt

from PIL import ImageFont, ImageDraw, Image
from pytesseract import Output
from google.colab.patches import cv2_imshow

In [ ]:
pytesseract.__version__

In [ ]:
cv2.__version__

In [ ]:
!mkdir tessdata
!wget -O ./tessdata/por.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/por.traineddata?raw=true
!wget -O ./tessdata/eng.traineddata https://github.com/tesseract-ocr/tessdata/blob/main/eng.traineddata?raw=true

#### Entendendo os dados

In [ ]:
projeto = "/content/text-recognize/Imagens/Projeto"
caminho = [os.path.join(projeto, f) for f in os.listdir(projeto)]
print(caminho)

In [ ]:
def mostrar(img):
  fig = plt.gcf()                                   # busca a figura atual
  fig.set_size_inches(20, 10)                       # define o tamanho
  plt.axis("off")                                   # remove a visualização dos eixos
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # fa a conversão das cores com o OpenCV
  plt.show()                                        # mosta a imagem

In [ ]:
for imagem in caminho:
  imagem = cv2.imread(imagem)
  mostrar(imagem)

In [ ]:
config_tesseract = "--tessdata-dir tessdata"

In [ ]:
def OCR_processa(img, config_tesseract):
  texto = pytesseract.image_to_string(img, lang="por", config=config_tesseract)
  return texto

### 2. Reconhecimento de texto

In [ ]:
texto_completo = ''
nome_txt = 'resultado_ocr.txt'

In [ ]:
for imagem in caminho:                                        # percorre as imagens no caminho
  img = cv2.imread(imagem)
  nome_imagem = os.path.split(imagem)[-1]                     # recebe os nomes e diretórios das imagens, quebrados, precisamos apenas do -1 (última posição do diretório)
  nome_divisao = '===================\n' + str(nome_imagem)   # divisão + nome da imagem que está sendo vista
  texto_completo = texto_completo + nome_divisao + '\n'       # recebe o texto completo + a divisão + /n para pular a linha
  texto = OCR_processa(img, config_tesseract)
  texto_completo += texto                                     # recebe o texto completo + o texto que foi reconhecido



In [ ]:
#texto_completo

In [ ]:
# Salvando o arquivo em txt
arquivo_txt = open(nome_txt, 'w+')        # a+ é para colocar no final do arquivo, w+ para sobre escrever o arquivo
arquivo_txt.write(texto_completo + '\n')  #passa o texto que quer adicionar
arquivo_txt.close()

### 3. Busca de ocorrências

#### Nos textos

In [ ]:
termo_pesquisa = 'learning' # @param {type:"string"}

In [ ]:
with open(nome_txt) as f:
  ocorrencias = [i.start() for i in re.finditer(termo_pesquisa, f.read())] # ocorrencia é uma lista.

In [ ]:
ocorrencias

Na listagem de imagens

In [ ]:
for imagem in caminho:
  img = cv2.imread(imagem)        # carrega a imagem
  nome_imagem = os.path.split(imagem)[-1]
  print('===================\n' + str(nome_imagem))   # separação + nome da imagem

  texto = OCR_processa(img, config_tesseract)         # usando o OCR

  ocorrencias = [i.start() for i in re.finditer(termo_pesquisa, texto)]
  if len(ocorrencias) == 0:
    print(f"O termo {termo_pesquisa} não foi encontrado.")
  else:
    if len(ocorrencias) == 1:
      palavra_ocorrencia = 'ocorrência'
    else:
      palavra_ocorrencia = 'ocorrências'

    print(f"Foi encontrado {len(ocorrencias)} {palavra_ocorrencia} do termo: {termo_pesquisa} \n")

### 4. Reconhecimento na imagem

In [ ]:
fonte_dir = '/content/text-recognize/Imagens/calibri.ttf'

In [ ]:
def escreve_texto(texto, x, y, img, fonte_dir, cor=(50, 50, 255), tamanho=16):
  fonte = ImageFont.truetype(fonte_dir, tamanho)
  img_pil = Image.fromarray(img)
  draw = ImageDraw.Draw(img_pil)
  draw.text((x, y-tamanho), texto, font = fonte, fill = cor)
  img = np.array(img_pil)
  return img

In [ ]:
min_conf = 30 #@param {type:"slider", min:0, max:100}

In [ ]:
def caixa_texto(i, resultado, img, cor=(255, 100, 0)):
  x = resultado["left"][i]
  y = resultado["top"][i]
  w = resultado["width"][i]
  h = resultado["height"][i]
  cv2.rectangle(img, (x, y), (x + w, y + h), cor, 2)
  return x, y, img

In [ ]:
def OCR_processa_imagem(img, termo_pesquisa, config_tesseract, min_conf):
  resultado = pytesseract.image_to_data(img, config=config_tesseract, lang='por', output_type=Output.DICT) #imagem para dados, que já fizemos anteriormente
  num_ocorrencias = 0 #inicializando como 0

  for i in range(0, len(resultado['text'])): # vai de 0 ao tamanho do número de valores do texto
    confianca = int(resultado['conf'][i]) # qual a confiança da detecção
    if confianca > min_conf: # se a confiança for maior que o valor mínimo, passa para a linha abaixo
      texto = resultado['text'][i] #texto será igual ao resultado text no momento i
      if termo_pesquisa in texto: #se o termo de pesquisa estiver no texto:
        x, y, img = caixa_texto(i, resultado, img, (0,0,255)) # faz a caixa de bounding box
        img = escreve_texto(texto, x, y, img, fonte_dir, (50,50,225), 14) #escreve o texto

        num_ocorrencias += 1 #faz a iteração no num de ocorrências e volta para o laço até acabar o texto
  return img, num_ocorrencias

In [ ]:
termo_pesquisa = 'learning'

for imagem in caminho:
  img = cv2.imread(imagem) #carrega a imagem
  img_original = img.copy() #cria uma cópia da imagem para não escrever em cima da original

  nome_imagem = os.path.split(imagem)[-1] # passa a imagem e acessa a última posição do diretório
  print('===================\n' + str(nome_imagem))  # separação + nome da imagem

  img, numero_ocorrencias = OCR_processa_imagem(img, termo_pesquisa, config_tesseract, min_conf) #usando o OCR_processa_imagem
  print('Número de ocorrências para {} em {}: {}'.format(termo_pesquisa, nome_imagem, numero_ocorrencias))
    #primeira chaves é para termo de pesquisa e a segunda nome das imagens e a terceira é o numeros de ocorrencias
  print('\n')

  mostrar(img)